In [1]:
import pandas as pd
import numpy as np
import pylogit
from scipy.special import logit
from datetime import timedelta
import statsmodels.formula.api as smf
from sklearn.metrics import brier_score_loss, mean_squared_error
pd.options.display.max_columns = 100

In [2]:
df = pd.read_csv('xc.csv', parse_dates = ['dt'])
df = df.loc[df.race_type == 'flat_race']

In [39]:
df.head()

,Unnamed: 0,race_id,stadium_id,distance_m,going,race_type,race_grade,dog_id,place,time,decimal_price,comment,box,kg,winner,dt,origTime,dnf
0,1,4418717,13025,515,-0.1,flat_race,A6,2176330,4,31.41,11.0,"BadlyBlkVW1,Crd3",5,27.75,0,2019-01-01,31.41,0
1,2,4418717,13025,515,-0.1,flat_race,A6,2207348,3,31.33,7.0,"CrdRnUp&1,Led2To 3/4",1,31.25,0,2019-01-01,31.33,0
2,3,4418717,13025,515,-0.1,flat_race,A6,2216661,5,31.49,7.0,"BadlyBlkWide1,Blk3",6,28.50,0,2019-01-01,31.49,0
3,4,4418717,13025,515,-0.1,flat_race,A6,2338001,2,31.31,3.5,"SAw,BBlk 1/4,Crowded3",4,35.50,0,2019-01-01,31.31,0
4,5,4418717,13025,515,-0.1,flat_race,A6,2342199,6,31.70,4.0,"EP,CrdRnUp&1& 1/4&3&4",3,34.00,0,2019-01-01,31.70,0


In [3]:
dups = df.groupby('race_id').box.agg(lambda x : len(x)-x.nunique())
dups = dups.loc[dups>0]

df = df.loc[~df.race_id.isin(dups.index)].copy()

win_choice = df.loc[df.winner == 1].sample(frac = 1.0).groupby('race_id').head(1).loc[:,['race_id','box']].copy()
win_choice['twinner'] = 1

dfm = df.merge(win_choice, on = ['race_id','box'], how = 'left')
dfm.twinner = dfm.twinner.fillna(0)
dfm = dfm.sort_values(['race_id','box'])

dfm = dfm.sort_values('dt')

In [4]:
# defining mmps and avg_mmps for each dog
startupTIME = 1.75
distDEFAULT = 400
distEXPONENT = 0.11
dfm['mps'] = dfm.time - startupTIME
dfm['mmps'] = dfm.distance_m/dfm.mps * (dfm.distance_m**distEXPONENT)/(distDEFAULT**distEXPONENT)

In [5]:
dfm['Unnamed: 0'] = dfm.index

In [6]:
# calculating avg mmps and weighted average mmps (using a halflife of 5 races)
dfm['avgmmps'] = dfm.groupby('dog_id').mmps.apply(lambda x: x.shift().expanding().mean())
dfm['mmps_ema'] = dfm.groupby('dog_id').mmps.apply(lambda x: x.ewm(halflife = 5).mean().shift())

In [ ]:
# alternate mmps_ema using days instead of races
group = dfm.groupby('dog_id').apply(lambda x: x.mmps.ewm(halflife = pd.Timedelta(10,unit = 'Days'), 
                                                         times = x.dt).mean().shift(1,fill_value=0)).reset_index()
group = group.rename(columns={'level_1': 'Unnamed: 0', 'mmps': 'mmps_ema'})
dfm = dfm.merge(group, on = ['dog_id', 'Unnamed: 0'])

In [7]:
dfm = pd.get_dummies(dfm, columns = ['stadium_id'])

In [8]:
# for 2.b - adding indicators for stadium_id
cols = dfm.columns[dfm.columns.str.contains('stadium_id')]
stadium_ema = dfm.groupby('dog_id')[cols].apply(lambda x: x.ewm(halflife = 5).mean().shift())

In [9]:
dfm_ind = dfm.copy()
dfm_ind[cols] = dfm[cols] - stadium_ema[cols]

In [10]:
# for 3 - adding indicators for certain comments
dfm_ind['Blk'] = np.where(dfm_ind.comment.str.contains('Blk|Baulk|Baulked'), 1, 0)
dfm_ind['Bmp'] = np.where(dfm_ind.comment.str.contains('Bmp|Bumped'), 1, 0)
dfm_ind['Stb'] = np.where(dfm_ind.comment.str.contains('Stb|Stumbled'), 1, 0)
dfm_ind['QAw'] = np.where(dfm_ind.comment.str.contains('QAw|QAway|QuickAway'), 1, 0)
dfm_ind['Ld'] = np.where(dfm_ind.comment.str.contains('Ld|Led'), 1, 0)
dfm_ind['Amp'] = dfm_ind.comment.str.count('&')

dfm_ind['Blk_ema'] = dfm_ind.groupby('dog_id').Blk.apply(lambda x: x.ewm(halflife = 5).mean().shift())
dfm_ind['Bmp_ema'] = dfm_ind.groupby('dog_id').Bmp.apply(lambda x: x.ewm(halflife = 5).mean().shift())
dfm_ind['Stb_ema'] = dfm_ind.groupby('dog_id').Stb.apply(lambda x: x.ewm(halflife = 5).mean().shift())
dfm_ind['QAw_ema'] = dfm_ind.groupby('dog_id').QAw.apply(lambda x: x.ewm(halflife = 5).mean().shift())
dfm_ind['Ld_ema'] = dfm_ind.groupby('dog_id').Ld.apply(lambda x: x.ewm(halflife = 5).mean().shift())
dfm_ind['Amp_ema'] = dfm_ind.groupby('dog_id').Amp.apply(lambda x: x.ewm(halflife = 5).mean().shift())

In [11]:
# filtering out races with less than 4 dogs
flattrack = dfm_ind.groupby('race_id').dog_id.agg(lambda x: x.count())
flattrack = flattrack.loc[flattrack >= 4]
dfmf = dfm_ind.loc[dfm_ind.race_id.isin(flattrack.index)].copy()

In [12]:
# add column to count number of prior races for each dog
dfmf['prior_races'] = dfmf.groupby('dog_id').cumcount()

In [13]:
# filter out races where all dogs did not have at least 3 prior races
priors = dfmf[dfmf['prior_races'] >= 3].groupby('race_id').prior_races.count()/dfmf.groupby('race_id').prior_races.count()
priors = priors.loc[priors == 1]
dfmfp = dfmf.loc[dfmf.race_id.isin(priors.index)].copy()

In [14]:
# create column with date of last race run by that dog
dfmfp['last_race'] = dfmfp.groupby('dog_id').dt.shift()

In [15]:
# filter out races where all dogs have not run a race in the past 90 days
ninety = dfmfp.loc[dfmfp.dt > dfmfp.last_race + timedelta(90)].race_id.unique()
df_final = dfmfp.loc[~dfmfp.race_id.isin(ninety)].copy()

In [40]:
train = df_final[df_final.dt.between('2019-07-01','2019-11-30')]
test = df_final[df_final.dt.between('2019-12-01','2020-01-31')]

In [78]:
# 2.a.i
model1 = smf.ols('mmps ~ mmps_ema', data = train).fit()

In [23]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mmps   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                 2.479e+05
Date:                Thu, 29 Apr 2021   Prob (F-statistic):               0.00
Time:                        14:20:46   Log-Likelihood:                -10326.
No. Observations:               85714   AIC:                         2.066e+04
Df Residuals:                   85712   BIC:                         2.068e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4990      0.034     14.731      0.000       0.433       0.565
mmps_ema       0.9715      0.002    497.944      0.000       0.968       0.975
==============================================================================
Omnibus:                     8939.881   Durbin-Watson:                   1.438
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28894.722
Skew:                          -0.536   Prob(JB):                         0.00
Kurtosis:                       5.634   Cond. No.                         633.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
ytrue = test.mmps
ypred = model1.predict(test)

In [26]:
# 2.a.ii.
print('Mean squared error on games between 12/01/2019 and 01/31/2020:', mean_squared_error(ytrue, ypred))

Mean squared error on games between 12/01/2019 and 01/31/2020: 0.07265294834042432


In [142]:
model_cols = '+'.join(train[cols].columns)
formula = 'mmps ~ mmps_ema +' + model_cols

In [140]:
# 2.b.i
model2 = smf.ols(formula = formula, data = train).fit()

In [141]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mmps   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.795
Method:                 Least Squares   F-statistic:                 1.584e+04
Date:                Thu, 29 Apr 2021   Prob (F-statistic):               0.00
Time:                        21:40:05   Log-Likelihood:                -633.55
No. Observations:               85714   AIC:                             1311.
Df Residuals:                   85692   BIC:                             1517.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.3849      0.030     12.697      0.000       0.325       0.444
mmps_ema             0.9782      0.002    560.266      0.000       0.975       0.982
stadium_id_13003    -0.0855      0.016     -5.513      0.000      -0.116      -0.055
stadium_id_13004     0.1393      0.011     12.708      0.000       0.118       0.161
stadium_id_13007    -0.5043      0.011    -43.961      0.000      -0.527      -0.482
stadium_id_13008    -0.0764      0.019     -4.007      0.000      -0.114      -0.039
stadium_id_13009    -0.2178      0.010    -22.804      0.000      -0.237      -0.199
stadium_id_13010     0.6534      0.011     61.984      0.000       0.633       0.674
stadium_id_13013     0.1920      0.024      7.907      0.000       0.144       0.240
stadium_id_13014     0.1702      0.015     11.017      0.000       0.140       0.200
stadium_id_13019    -0.1029      0.010     -9.810      0.000      -0.123      -0.082
stadium_id_13020    -0.1284      0.018     -7.279      0.000      -0.163      -0.094
stadium_id_13021    -0.0004      0.044     -0.009      0.992      -0.087       0.086
stadium_id_13023     0.1094      0.015      7.081      0.000       0.079       0.140
stadium_id_13025     0.5451      0.011     49.104      0.000       0.523       0.567
stadium_id_13026    -0.3743      0.010    -37.002      0.000      -0.394      -0.354
stadium_id_13035     0.3276      0.017     18.840      0.000       0.294       0.362
stadium_id_13037     0.3659      0.011     33.298      0.000       0.344       0.387
stadium_id_13043     0.0639      0.012      5.160      0.000       0.040       0.088
stadium_id_13048    -0.8013      0.011    -70.714      0.000      -0.824      -0.779
stadium_id_13053     0.1359      0.030      4.471      0.000       0.076       0.195
stadium_id_13059     0.1910      0.024      7.875      0.000       0.143       0.239
stadium_id_13061    -0.6025      0.012    -48.970      0.000      -0.627      -0.578
==============================================================================
Omnibus:                     7933.062   Durbin-Watson:                   1.439
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16123.363
Skew:                          -0.611   Prob(JB):                         0.00
Kurtosis:                       4.738   Cond. No.                     3.47e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.15e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [213]:
ytrue = test.mmps
ypred = model2.predict(test)

In [214]:
# 2.b.ii.
print('Mean squared error on games between 12/01/2019 and 01/31/2020:', mean_squared_error(ytrue, ypred))

Mean squared error on games between 12/01/2019 and 01/31/2020: 0.06470119367888517


In [41]:
# 3.a.
com_cols = 'Blk_ema + Bmp_ema + Stb_ema + QAw_ema + Ld_ema + Amp_ema'
stad_cols = '+'.join(train[cols].columns)
formula = 'mmps ~ mmps_ema +' + stad_cols + '+' + com_cols

In [42]:
model3 = smf.ols(formula = formula, data = train).fit()

In [43]:
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mmps   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                 1.244e+04
Date:                Mon, 10 May 2021   Prob (F-statistic):               0.00
Time:                        20:13:31   Log-Likelihood:                -283.44
No. Observations:               85714   AIC:                             622.9
Df Residuals:                   85686   BIC:                             884.9
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.2264      0.032      6.985      0.000       0.163       0.290
mmps_ema             0.9862      0.002    530.621      0.000       0.983       0.990
stadium_id_13003    -0.0874      0.015     -5.658      0.000      -0.118      -0.057
stadium_id_13004     0.1434      0.011     13.132      0.000       0.122       0.165
stadium_id_13007    -0.5043      0.011    -44.134      0.000      -0.527      -0.482
stadium_id_13008    -0.0771      0.019     -4.058      0.000      -0.114      -0.040
stadium_id_13009    -0.2161      0.010    -22.718      0.000      -0.235      -0.197
stadium_id_13010     0.6531      0.011     62.202      0.000       0.633       0.674
stadium_id_13013     0.1967      0.024      8.134      0.000       0.149       0.244
stadium_id_13014     0.1663      0.015     10.806      0.000       0.136       0.196
stadium_id_13019    -0.1029      0.010     -9.848      0.000      -0.123      -0.082
stadium_id_13020    -0.1258      0.018     -7.163      0.000      -0.160      -0.091
stadium_id_13021    -0.0070      0.044     -0.160      0.873      -0.093       0.079
stadium_id_13023     0.1080      0.015      7.013      0.000       0.078       0.138
stadium_id_13025     0.5494      0.011     49.686      0.000       0.528       0.571
stadium_id_13026    -0.3730      0.010    -37.014      0.000      -0.393      -0.353
stadium_id_13035     0.3229      0.017     18.642      0.000       0.289       0.357
stadium_id_13037     0.3655      0.011     33.395      0.000       0.344       0.387
stadium_id_13043     0.0654      0.012      5.306      0.000       0.041       0.090
stadium_id_13048    -0.8017      0.011    -71.023      0.000      -0.824      -0.780
stadium_id_13053     0.1378      0.030      4.552      0.000       0.078       0.197
stadium_id_13059     0.1890      0.024      7.821      0.000       0.142       0.236
stadium_id_13061    -0.6023      0.012    -49.152      0.000      -0.626      -0.578
Blk_ema              0.0645      0.008      7.931      0.000       0.049       0.081
Bmp_ema              0.0495      0.005      9.035      0.000       0.039       0.060
Stb_ema              0.1524      0.030      5.052      0.000       0.093       0.212
QAw_ema             -0.0600      0.007     -9.018      0.000      -0.073      -0.047
Ld_ema               0.0015      0.005      0.305      0.760      -0.008       0.011
Amp_ema              0.0880      0.005     16.073      0.000       0.077       0.099
==============================================================================
Omnibus:                     7815.450   Durbin-Watson:                   1.439
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15690.583
Skew:                          -0.607   Prob(JB):                         0.00
Kurtosis:                       4.708   Cond. N

In [44]:
ytrue = test.mmps
ypred = model3.predict(test)

In [216]:
# 3.b.
print('Mean squared error on games between 12/01/2019 and 01/31/2020:', mean_squared_error(ytrue, ypred))

Mean squared error on games between 12/01/2019 and 01/31/2020: 0.06455910011979972


In [16]:
def mlogit(formula, df, obs_id, alt_id) :
    """
    df must be sorted by obs_id to use this function
    """
    from patsy import dmatrices
    from collections import OrderedDict
    import pylogit as pl
    import numpy as np
    data = dmatrices(formula+' -1', df, return_type = "dataframe")
    all_alts = sorted(df.loc[:,alt_id].unique())
    
    spec = OrderedDict()
    names = OrderedDict()
    spec['intercept'] = all_alts[1:]
    names['intercept'] = [f'(Intercept):{alt}' for alt in all_alts[1:]]
    c_names = data[1].design_info.column_names
    choice = data[0].design_info.column_names[0]
    data[1][choice] = data[0]
    data[1][[obs_id,alt_id]] = df[[obs_id,alt_id]]    
    for c in c_names :
        spec[c] = [all_alts]
        names[c] = [c]
    model = pl.create_choice_model(data = data[1], alt_id_col = alt_id, obs_id_col = obs_id, 
                                   choice_col = choice, specification = spec, names = names, model_type = 'MNL')
    model.fit_mle(np.zeros(model.design.shape[1]), method = 'Powell')
    return model

In [157]:
# 4.a.i.
# making mmps predictions on whole dataset and adding as a feature for new twinner model
mmps_forecast = model3.predict(df_final)
df_final4 = df_final.copy()
df_final4['mmps_forecast'] = mmps_forecast

In [158]:
# shifted decimal price and going
df_final4['decimal_shift'] = df_final4.groupby('dog_id').decimal_price.apply(lambda x: x.ewm(halflife = 5).mean().shift())
df_final4['going_shift'] = df_final4.groupby('dog_id').going.apply(lambda x: x.ewm(halflife = 5).mean().shift())

In [159]:
# conversion of decimal price to prob
decimal_sum = df_final4.groupby('race_id', as_index = False).decimal_price.apply(lambda x: x.sum())
decimal_sum = decimal_sum.rename(columns = {'decimal_price':'decimal_sum'})

df_final4 = df_final4.merge(decimal_sum, on = 'race_id')

In [171]:
df_final4['market_prob'] = logit(df_final4.decimal_price/df_final4.decimal_sum)

In [162]:
train = df_final4[df_final4.dt.between('2019-07-01','2020-01-31')].sort_values(['race_id','box'])
test = df_final4[df_final4.dt > '2020-01-31'].sort_values(['race_id','box'])

In [50]:
model4 = mlogit('twinner ~ mmps_forecast', train, 'race_id', 'box')

Log-likelihood at zero: -36,672.1279
Initial Log-likelihood: -36,672.1279


C:\Users\sanjs\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:519: RuntimeWarning: Method Powell does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,
C:\Users\sanjs\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:524: RuntimeWarning: Method Powell does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
C:\Users\sanjs\anaconda3\lib\site-packages\pylogit\estimation.py:678: OptimizeWarning: Unknown solver options: gtol
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 4.15 seconds.
Final log-likelihood: -35,740.7729


In [51]:
model4.get_statsmodels_summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               20,721
Model:             Multinomial Logit Model   Df Residuals:                   20,715
Method:                                MLE   Df Model:                            6
Date:                     Mon, 10 May 2021   Pseudo R-squ.:                   0.025
Time:                             20:18:01   Pseudo R-bar-squ.:               0.025
AIC:                            71,493.546   Log-Likelihood:            -35,740.773
BIC:                            71,541.179   LL-Null:                   -36,672.128
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
(Intercept):2    -0.0506      0.024     -2.089      0.037      -0.098      -0.003
(Intercept):3     0.0357      0.024      1.489      0.136      -0.011       0.083
(Intercept):4    -0.0059      0.024     -0.244      0.808      -0.053       0.041
(Intercept):5    -0.0950      0.025     -3.848      0.000      -0.143      -0.047
(Intercept):6     0.0405      0.024      1.687      0.092      -0.007       0.088
mmps_forecast     3.3922      0.082     41.355      0.000       3.231       3.553
=================================================================================
"""

In [52]:
ytrue = test.twinner
ypreds = model4.predict(test)

In [53]:
# 4.a.ii.
print('Brier score loss on games after 01/31/2020:', brier_score_loss(ytrue, ypreds))

Brier score loss on games after 01/31/2020: 0.13861083828837706


In [118]:
# 4.b.i.
model5 = mlogit('twinner ~ mmps_forecast + decimal_shift + going_shift', train, 'race_id', 'box')

Log-likelihood at zero: -34,576.2017
Initial Log-likelihood: -34,576.2017


C:\Users\sanjs\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:519: RuntimeWarning: Method Powell does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,
C:\Users\sanjs\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:524: RuntimeWarning: Method Powell does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
C:\Users\sanjs\anaconda3\lib\site-packages\pylogit\estimation.py:678: OptimizeWarning: Unknown solver options: gtol
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 6.23 seconds.
Final log-likelihood: -33,515.3529


In [119]:
model5.get_statsmodels_summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               20,719
Model:             Multinomial Logit Model   Df Residuals:                   20,711
Method:                                MLE   Df Model:                            8
Date:                     Mon, 10 May 2021   Pseudo R-squ.:                   0.031
Time:                             21:28:30   Pseudo R-bar-squ.:               0.030
AIC:                            67,046.706   Log-Likelihood:            -33,515.353
BIC:                            67,110.216   LL-Null:                   -34,576.202
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
(Intercept):2    -0.0539      0.024     -2.213      0.027      -0.102      -0.006
(Intercept):3     0.0243      0.024      1.002      0.317      -0.023       0.072
(Intercept):4    -0.0014      0.024     -0.058      0.954      -0.049       0.046
(Intercept):5    -0.0858      0.025     -3.463      0.001      -0.134      -0.037
(Intercept):6     0.0530      0.024      2.205      0.027       0.006       0.100
mmps_forecast     3.3097      0.085     38.882      0.000       3.143       3.477
decimal_shift    -0.0849      0.005    -17.222      0.000      -0.095      -0.075
going_shift      -0.0883      0.157     -0.561      0.575      -0.397       0.220
=================================================================================
"""

In [120]:
ytrue = test.twinner
ypreds = model5.predict(test)

In [121]:
# 4.b.ii.
print('Brier score loss on games after 01/31/2020:', brier_score_loss(ytrue[~np.isnan(ypreds)], ypreds[~np.isnan(ypreds)]))

Brier score loss on games after 01/31/2020: 0.13799497683809794


In [172]:
# 4.c.i.
twinner_forecast = model4.predict(df_final4)
df_final4['twinner_forecast'] = logit(twinner_forecast)

In [173]:
train = df_final4[df_final4.dt.between('2019-07-01','2020-01-31')].sort_values(['race_id','box'])
test = df_final4[df_final4.dt > '2020-01-31'].sort_values(['race_id','box'])

In [174]:
model6 = mlogit('twinner ~ twinner_forecast + market_prob', train, 'race_id', 'box')

Log-likelihood at zero: -36,672.1279
Initial Log-likelihood: -36,672.1279


C:\Users\sanjs\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:519: RuntimeWarning: Method Powell does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,
C:\Users\sanjs\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:524: RuntimeWarning: Method Powell does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
C:\Users\sanjs\anaconda3\lib\site-packages\pylogit\estimation.py:678: OptimizeWarning: Unknown solver options: gtol
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 13.12 seconds.
Final log-likelihood: -33,579.7385


In [167]:
model6.get_statsmodels_summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               20,721
Model:             Multinomial Logit Model   Df Residuals:                   20,714
Method:                                MLE   Df Model:                            7
Date:                     Mon, 10 May 2021   Pseudo R-squ.:                   0.084
Time:                             23:58:10   Pseudo R-bar-squ.:               0.084
AIC:                            67,173.477   Log-Likelihood:            -33,579.739
BIC:                            67,229.049   LL-Null:                   -36,672.128
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
(Intercept):2              -0.0018      0.025     -0.073      0.942      -0.051       0.047
(Intercept):3               0.0067      0.025      0.272      0.785      -0.041       0.055
(Intercept):4               0.0270      0.025      1.092      0.275      -0.022       0.076
(Intercept):5               0.0242      0.025      0.953      0.341      -0.026       0.074
(Intercept):6               0.0516      0.025      2.094      0.036       0.003       0.100
logit(twinner_forecast)     0.1436      0.022      6.512      0.000       0.100       0.187
logit(market_prob)         -1.0391      0.017    -62.127      0.000      -1.072      -1.006
===========================================================================================
"""

In [176]:
ytrue = test.twinner
ypreds = model6.predict(test)

In [177]:
# 4.c.ii.
print('Brier score loss on games after 01/31/2020:', brier_score_loss(ytrue[~np.isnan(ypreds)], ypreds[~np.isnan(ypreds)]))

Brier score loss on games after 01/31/2020: 0.13156691078722232
